In [ ]:
!pip install openpyxl
!pip install BeautifulSoup
!pip install json

In [296]:
import requests
import json
import time
import copy
import openpyxl
from bs4 import BeautifulSoup as bs
from tqdm import tqdm, trange

In [297]:
SIDO_DICTONARY = {'서울':'11', '인천':'28', '세종':'36', '대전':'30', '광주':'29', '울산':'31', 
                  '대구':'27', '부산':'26', '경기':'41', '강원':'42', '충북':'43', '충남':'44', 
                  '충북':'43', '경북':'47', '전북':'45', '전남':'46', '경남':'48'}
# 시에서 구을 검색할때 코드
#BJD_CODE = 'SGG'
#구에서 동을 검색할때 코드
#BJD_CODE = 'EMD'

In [298]:
urlSido = 'http://www.k-apt.go.kr/cmmn/bjd/getBjdList.do?' # 동까지 조회 할때 사용하는 url
urlApt = 'http://www.k-apt.go.kr/kaptinfo/getKaptList.do?'  # 아파트리스트 가져올때 사용하는 url

# urlMonth = 'http://www.k-apt.go.kr/apiinfo/openCostMonthUseAve.do?' # 월별 에너지량
# url = 'http://www.k-apt.go.kr/apiinfo/openCostEnergyUseStatistics.do?' # 에너지사용 통계
# url = 'http://www.k-apt.go.kr/apiinfo/openCostYearUseAve.do' # 연도 에너리량

In [ ]:
# 페이지로 응답오는 화면인경우 리다이렉트로 실해앟게 되어 있음..
# 크롤링을 막으려 하는건지 코딩을 이따구로 하는건지 확인 못하겠네...
# data = {'go_url':'/kaptinfo/openkaptinfo.do','bjd_code': '1168010300', 'kapt_code': 'A13593908', 'search_date': '202203'}  # 서울시 강남구 개포동
# response = web_request(method_name='POST', url='http://www.k-apt.go.kr/cmmn/selectKapt.do?', dict_data=data)
#http://www.k-apt.go.kr/cmmn/selectKapt.do
#http://www.k-apt.go.kr/kaptinfo/openkaptinfo.do
# '/kaptinfo/openkaptinfo.do? #우리단지 기본정보
# '/kaptinfo/openkaptinfoBid.do?'# 우리단지 입찰공고
# '/kaptinfo/openKaptMng.do?' #관리시설정보
# '/kaptinfo/openKaptSetday.do?' #주택인도일정보
# '/kaptinfo/openKaptGrEng.do?' #녹색/에너지정보
# '/kaptinfo/openKaptMngRule.do?' #관리규약
# '/kaptinfo/openKaptRep.do?' #입주자대표회의 구성인원

#<form id="cmnmMapFrm" method="post" action="/cmmn/selectKapt.do">
	#<input type="hidden" value="" name="go_url" id="go_url"> 
	#<input type="hidden" value="" name="bjd_code" id="bjd_code"> 
	#<input type="hidden" value="" name="kapt_code" id="kapt_code">
	#<input type="hidden" value="" name="search_date" id="search_date">
#</form>


In [299]:
def web_request(method_name, url, dict_data, is_urlencoded=True):
    """Web GET or POST request를 호출 후 그 결과를 dict형으로 반환 """
    method_name = method_name.upper()  # 메소드이름을 대문자로 바꾼다
    if method_name not in ('GET', 'POST'):
        raise Exception('method_name is GET or POST plz...')

    if method_name == 'GET':  # GET방식인 경우
        response = requests.get(url=url, params=dict_data)
    elif method_name == 'POST':  # POST방식인 경우
        if is_urlencoded is True:
            response = requests.post(url=url, data=dict_data,
                                     headers={'Content-Type': 'application/x-www-form-urlencoded'})
        else:
            response = requests.post(url=url, data=json.dumps(dict_data), headers={'Content-Type': 'application/json'})

    dict_meta = {'status_code': response.status_code, 'ok': response.ok, 'encoding': response.encoding,
                 'Content-Type': response.headers['Content-Type']}
    if 'json' in str(response.headers['Content-Type']):  # JSON 형태인 경우
        return {**dict_meta, **response.json()}
    else:  # 문자열 형태인 경우
        return {**dict_meta, **{'text': response.text}}

In [300]:
def getSidoList(paramCode, paramGbn):
    data = {'bjd_code': paramCode, 'bjd_gbn': paramGbn}  # 서울시
    # data = {'bjd_code': '11', 'bjd_gbn':'SGG'}           # 서울시
    # data = {'bjd_code': '11680', 'bjd_gbn':'EMD'}         # 서울시 강남구

    response = web_request(method_name='POST', url=urlSido, dict_data=data)
    if response['ok'] == True:
        # 성공 응답 시 액션
        result = json.loads(response['text'])
        return result
    else:
        # 실패 응답 시 액션
        pass

In [301]:
def getAptList(paramCode, srchDate):
    # data = {'bjd_code': '1168010300'}         # 서울시 강남구 개포동
    data = {'bjd_code': paramCode, 'search_date': srchDate}  # 서울시 강남구 개포동
    response = web_request(method_name='POST', url=urlApt, dict_data=data)
    if response['ok'] == True:
        # 성공 응답 시 액션
        result = json.loads(response['text'])
        return result
    else:
        pass

In [302]:
def getRedirectInfo(parent_url, child_url, bjd_code, kapt_code, search_date):
    data = {'go_url':child_url,'bjd_code': bjd_code, 'kapt_code': kapt_code, 'search_date': search_date}  # 서울시 강남구 개포동
    return web_request(method_name='POST', url=parent_url, dict_data=data)

In [303]:
# 우리단지 기본정보
#data = {'go_url':'/kaptinfo/openkaptinfo.do','bjd_code': '1168010300', 'kapt_code': 'A13593908', 'search_date': '202203'}  # 서울시 강남구 개포동
#danji_res = web_request(method_name='POST', url='http://www.k-apt.go.kr/cmmn/selectKapt.do?', dict_data=data)
def getDanjiInfo(bjd_code, kapt_code, search_date):
    result=[]
    danji_res = getRedirectInfo('http://www.k-apt.go.kr/cmmn/selectKapt.do?', '/kaptinfo/openkaptinfo.do', bjd_code, kapt_code, search_date)
    #danji_res = getRedirectInfo('http://www.k-apt.go.kr/cmmn/selectKapt.do?', '/kaptinfo/openkaptinfo.do', '1168010300', 'A13593908', '202203')
    danji_soup = bs(danji_res['text'], "html.parser")

    elements = danji_soup.find("div", "subCbox")
    td_values = elements.find_all('td')
    tr_values = elements.find_all('tr')
    script_values = elements.find("script").text
    ADDRESS_KEY = 'arrAddr.push'
    addr_val = ''

    for item in script_values.split('\n'):
        # 첫번째 값은 법정동주소고 두번째 값은 도로명주소이다
        if ADDRESS_KEY in item:
            addr_val= item[item.find('"')+1:-4]
            break
    #for idx, item in enumerate(elements):            # enumerate를 사용하면 해당 값의 인덱스를 알 수 있다.
    #    ths = item.find_all('th')
    #    tds = item.find_all('td')
    #    print(tds)
    
    result.append(0) # index
    result.append(td_values[0].text.strip()) # 명칭
    result.append(addr_val) #법정동주소    
    result.append(td_values[4].text.strip()) #분양형태
    result.append(td_values[5].text.strip()) #관리방식
    result.append(td_values[6].text.strip()) #난방방식
    result.append(td_values[7].text.strip()) #복도유형
    result.append(td_values[8].text.strip()) #연면적
    result.append(td_values[10].text.strip().split('\n')[0][:-4].replace(" ", "")) #동수/세대수
    result.append(td_values[10].text.strip().split('\n')[2]) #주거전용면적
    result.append(td_values[12].text.strip().replace(" ", "")) #시공사/시행사
    result.append(td_values[16].text.strip().split('\n')[0][td_values[16].text.strip().split('\n')[0].find(':')+1:-2].strip()) #면적별 세대현황(전용면적 기준)
    result.append(td_values[16].text.strip().split('\n')[1][td_values[16].text.strip().split('\n')[1].find(':')+1:-2].strip()) #면적별 세대현황(전용면적 기준)
    result.append(td_values[16].text.strip().split('\n')[2][td_values[16].text.strip().split('\n')[2].find(':')+1:-2].strip()) #면적별 세대현황(전용면적 기준)
    result.append(td_values[16].text.strip().split('\n')[3][td_values[16].text.strip().split('\n')[3].find(':')+1:-2].strip()) #면적별 세대현황(전용면적 기준)
    
    return result

In [304]:
# 관리시설정보
# 원래 실행순서 http://www.k-apt.go.kr/cmmn/selectKapt.do -> redirect : /kaptinfo/openKaptMng.do -> 상세정보는  /kaptinfo/getKaptInfo_detail.do 에서 호출
# 따라서 마지막 정보만 필요하기때문에 /kaptinfo/getKaptInfo_detail.do 만 호출해서 사용 만약에 추가필드가 필요하면 화면정보를 다시 불러와서 key value를 파악해야함

#manage_res = getRedirectInfo('http://www.k-apt.go.kr/cmmn/selectKapt.do?', '/kaptinfo/openKaptMng.do', '1168010300', 'A13593908', '202203')
#manage_soup = bs(manage_res['text'], "html.parser")
def manageInfo(kapt_code):
    result = []
    manage_data = {'kapt_code': kapt_code}  # 서울시 강남구 개포동
    manage_res = web_request(method_name='POST', url='http://www.k-apt.go.kr/kaptinfo/getKaptInfo_detail.do?', dict_data=manage_data)

    json_object = json.loads(manage_res['text'])
    result.append(json_object['resultMap_kapt']['CODE_ECON']) #세대전기방식
    result.append(json_object['resultMap_kapt']['KAPTD_ECNT_TOTAL']) #승강기대수
    result.append(json_object['resultMap_kapt']['KAPTD_PCNT']) # 지상주차
    result.append(json_object['resultMap_kapt']['KAPTD_PCNTU']) # 지하주차
    return result

In [305]:
# 1. 시에서 전체구를 조회하는 리스트를 만든다.
# 시에서 구를 검색하니 SGG값을 이용
SIDO_KEY = SIDO_DICTONARY['서울']
SIDO_CODE = 'SGG'

print('Start guList')
GU_LIST = getSidoList(SIDO_KEY, SIDO_CODE)
print('End guList')

Start guList
End guList


In [310]:
with open('./sido_data.json','w', encoding='utf-8') as f:
  json.dump(GU_LIST, f, ensure_ascii=False, indent=4)

In [311]:
#2. 각구별로 돌아가면서 동을 출력하는 리스트를 만든다.
# 구에서 동을 검색하니 EMD값을 이용
DONG_LIST = []
DONG_CODE = 'EMD'

print('Start dongList')
for gu in GU_LIST['resultList']:
    time.sleep(5)
    gu['CODE'] = SIDO_KEY + gu['CODE']
    dongList = getSidoList(gu['CODE'], DONG_CODE)
    for dong in dongList['resultList']:
        dong['CODE'] = gu['CODE'] + dong['CODE']
        DONG_LIST.append(dong)
print('End dongList')

Start dongList
End dongList


In [312]:
with open('./dong_data.json','w', encoding='utf-8') as f:
  json.dump(DONG_LIST, f, ensure_ascii=False, indent=4)

In [314]:
#3. 각동별로 아파트 정보를 년월기준으로 조회한다.
# 기준월을 세팅한다.
APT_LIST = []
SEARCH_DATE = '202203'

print('Start aptList')
for dong in DONG_LIST:
    time.sleep(5)
    temp = getAptList(dong['CODE'], SEARCH_DATE)
    for aptInfo in temp['resultList']:
        APT_LIST.append(aptInfo)
print('End aptList')

Start aptList


JSONDecodeError: Expecting value: line 3 column 1 (char 4)

In [ ]:
with open('./apt_data.json','w', encoding='utf-8') as f:
  json.dump(APT_LIST, f, ensure_ascii=False, indent=4)

In [ ]:
LIST_FOR_EXCEL = []

print('Start get APT engeryInfo')
for aptInfo in APT_LIST:
    # 단지정보 리스트를 받아온다
    # 관리시설정보 리스트를 받아온다
    # 두리스트를 합치고 LIST_FOR_EXCEL 에 append한다
    result = getDanjiInfo(aptInfo['BJD_CODE'], aptInfo['KAPT_CODE'],SEARCH_DATE)
    result2 = manageInfo(aptInfo['KAPT_CODE'],)
    result.extend(result2)
    LIST_FOR_EXCEL.append(result)
    time.sleep(5)
    #bjdList = aptInfo['BJD_NAME'].split(' ')
    #createResultByExcel.inputDataInExcel(bjdList[0],bjdList[1],bjdList[2],aptInfo['KAPT_NAME'], inqResult)
print('End get APT engeryInfo')

In [295]:
wb = openpyxl.load_workbook('크롤링부탁1.xlsx')
sheetA = wb.get_sheet_by_name('Sheet1')

cellListA =  [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,              16,17,18,19,20]
for item in LIST_FOR_EXCEL:
    sheetA.append(item)
wb.save('공동주택에너지.xlsx')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_16672\3361208224.py:2: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  sheetA = wb.get_sheet_by_name('Sheet1')
